In [7]:
# python packages
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install nltk
!pip install beautifulsoup4
!pip install textblob

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.8 MB 10.1 MB/s eta 0:00:02
     ------------- -------------------------- 4.2/12.8 MB 10.5 MB/s eta 0:00:01
     -------------------- ------------------- 6.6/12.8 MB 10.3 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 10.5 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.8 MB 10.2 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 10.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/refs/heads/master/twitter_sentiment.csv', header=None, index_col=0)
df.head()

,1,2,3
0,,,
2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2401,Borderlands,Positive,im coming on borderlands and i will murder you...
2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [10]:
df = df[[2,3]].reset_index(drop=True)
df.head()

,2,3
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [11]:
df.columns = ['sentiment', 'text']
df.head()

,sentiment,text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75682 entries, 0 to 75681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  75682 non-null  object
 1   text       74996 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


#### Data Analysis

In [14]:
df.isnull().sum()

sentiment      0
text         686
dtype: int64

In [22]:
# we need to filter out all the NULL texts
df.dropna(inplace=True)
df['text'].apply(len).value_counts()

text
70     611
3      566
2      500
36     485
57     480
      ... 
418      1
434      1
394      1
427      1
374      1
Name: count, Length: 403, dtype: int64

In [24]:
sum(df['text'].apply(len)>5), sum(df['text'].apply(len)<=5)

(72883, 2113)

In [31]:
df.sample(10)

,sentiment,text
21027,Positive,@Warcraft leveling change looks very interesti...
66128,Negative,Are they being sued for their baby powder? Nev...
7183,Irrelevant,I'm insanely bored. Go live with some random O...
56617,Positive,It is not the first time that the EU Commissio...
666,Positive,So I'm probably going to log off social media ...
54532,Negative,@Treyarch Call of Duty Black Ops 3 is the wors...
53185,Neutral,RED DEATH REDEMPTION 2nd
13947,Negative,@NBA2K y’all fuck up cause now it’s back to du...
16919,Irrelevant,I’m Going Google this again @imjustbait is the...
37504,Irrelevant,I Cheer on this @Deliver_HS in the final top p...


In [71]:
df['sentiment'].value_counts()

sentiment
Negative      22624
Positive      20932
Neutral       18393
Irrelevant    13047
Name: count, dtype: int64

In [73]:
!pip install text-preprocessing

   ---------------------------------------- 0.0/62.6 MB ? eta -:--:--
   - -------------------------------------- 2.1/62.6 MB 10.7 MB/s eta 0:00:06
   -- ------------------------------------- 4.2/62.6 MB 10.5 MB/s eta 0:00:06
   --- ------------------------------------ 6.0/62.6 MB 10.2 MB/s eta 0:00:06
   ----- ---------------------------------- 8.1/62.6 MB 10.3 MB/s eta 0:00:06
   ------ --------------------------------- 10.0/62.6 MB 9.8 MB/s eta 0:00:06
   ------- -------------------------------- 12.1/62.6 MB 9.9 MB/s eta 0:00:06
   --------- ------------------------------ 14.2/62.6 MB 10.0 MB/s eta 0:00:05
   ---------- ----------------------------- 16.0/62.6 MB 10.0 MB/s eta 0:00:05
   ----------- ---------------------------- 18.4/62.6 MB 10.1 MB/s eta 0:00:05
   ------------- -------------------------- 20.4/62.6 MB 10.1 MB/s eta 0:00:05
   -------------- ------------------------- 22.8/62.6 MB 10.2 MB/s eta 0:00:04
   ---------------- ----------------------- 25.2/62.6 MB 10.3 MB/s 

In [81]:
from text_preprocessing import preprocess_text
df['text'] = df['text'].apply(lambda x: x.lower())

In [83]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state = 0)
X_train.shape, X_test.shape

((59996,), (15000,))

#### Model Building and Testing

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [98]:
clf = Pipeline([('tfid', TfidfVectorizer()), ('rfc', RandomForestClassifier(n_jobs=-1))])
clf.fit(X_train, y_train)

Pipeline(steps=[('tfid', TfidfVectorizer()),
                ('rfc', RandomForestClassifier(n_jobs=-1))])

In [111]:
# evaluation
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  Irrelevant       0.98      0.85      0.91      2675
    Negative       0.92      0.95      0.93      4489
     Neutral       0.92      0.93      0.92      3742
    Positive       0.90      0.94      0.92      4094

    accuracy                           0.92     15000
   macro avg       0.93      0.91      0.92     15000
weighted avg       0.92      0.92      0.92     15000

